In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv("marc.env") #Here goes the credentials for your Database
                        #Use a .env with the structure below

db_username = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

engine = create_engine(f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}")

query = "SELECT * FROM candidates_clean"

df = pd.read_sql_query(query, engine)

In [2]:
df.head

<bound method NDFrame.head of        First Name   Last Name                       Email Application Date  \
0      Bernadette   Langworth         leonard91@yahoo.com       2021-02-26   
1          Camryn    Reynolds         zelda56@hotmail.com       2021-09-09   
2           Larue      Spinka    okey_schultz41@gmail.com       2020-04-14   
3            Arch      Spinka      elvera_kulas@yahoo.com       2020-10-01   
4           Larue  Altenwerth   minnie.gislason@gmail.com       2020-05-20   
...           ...         ...                         ...              ...   
49995     Bethany     Shields  rocky_mitchell@hotmail.com       2022-01-09   
49996         Era  Swaniawski    dolores.roob@hotmail.com       2020-06-02   
49997      Martin       Lakin   savanah.stracke@gmail.com       2018-12-15   
49998       Aliya   Abernathy  vivienne.fritsch@yahoo.com       2020-05-30   
49999     Coleman      Wisozk   abigayle.crooks@yahoo.com       2022-06-13   

                  Country  YOE  S

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   First Name                 50000 non-null  object
 1   Last Name                  50000 non-null  object
 2   Email                      50000 non-null  object
 3   Application Date           50000 non-null  object
 4   Country                    50000 non-null  object
 5   YOE                        50000 non-null  int64 
 6   Seniority                  50000 non-null  object
 7   Technology                 50000 non-null  object
 8   Code Challenge Score       50000 non-null  int64 
 9   Technical Interview Score  50000 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 3.8+ MB
None


EDA

Hired

We need to create a column that determinates if the candidate was hired. According to the challenge, a candidate is hired when he has both scores greater than or equal to 7.

In [4]:
def is_hired(row):
    if row["Code Challenge Score"] >= 7 and row["Technical Interview Score"] >= 7:
        return True
    elif row["Code Challenge Score"] < 7 and row["Technical Interview Score"] < 7:
        return False
    else:
        return False


df["Hired"] = df.apply(is_hired, axis=1)

print(df.info()) #The new column is bool, this is because data with only two unique values (Yes/No, Used/New, etc.) 
                 #can be better stored as boolean, it improves performance and reduces storage requirements

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   First Name                 50000 non-null  object
 1   Last Name                  50000 non-null  object
 2   Email                      50000 non-null  object
 3   Application Date           50000 non-null  object
 4   Country                    50000 non-null  object
 5   YOE                        50000 non-null  int64 
 6   Seniority                  50000 non-null  object
 7   Technology                 50000 non-null  object
 8   Code Challenge Score       50000 non-null  int64 
 9   Technical Interview Score  50000 non-null  int64 
 10  Hired                      50000 non-null  bool  
dtypes: bool(1), int64(3), object(7)
memory usage: 3.9+ MB
None


Variable description
Lets see more info about the variables according to their type

In [5]:
#numerical
numerical_summary = df.describe()
numerical_summary

,YOE,Code Challenge Score,Technical Interview Score
count,50000.000000,50000.000000,50000.000000
mean,15.286980,4.996400,5.003880
std,8.830652,3.166896,3.165082
min,0.000000,0.000000,0.000000
25%,8.000000,2.000000,2.000000
50%,15.000000,5.000000,5.000000
75%,23.000000,8.000000,8.000000
max,30.000000,10.000000,10.000000


In [6]:
#categorical (Object and bool)
categorical_summary = df.describe(include=['object','bool'])
categorical_summary

,First Name,Last Name,Email,Application Date,Country,Seniority,Technology,Hired
count,50000,50000,50000,50000,50000,50000,50000,50000
unique,3007,474,49833,1646,244,7,24,2
top,Sarai,Murazik,marianne31@yahoo.com,2020-07-07,Malawi,Intern,Game Development,False
freq,33,138,3,50,242,7255,3818,43302


Cleaning
Searching for posible cleaning process, lets check Email who could be a option for candidates id.

In [16]:
print(df['Email'].value_counts())

Email
marianne31@yahoo.com          3
fern70@gmail.com              3
isaiah24@yahoo.com            2
missouri65@yahoo.com          2
kelton91@hotmail.com          2
                             ..
rocky_mitchell@hotmail.com    1
dolores.roob@hotmail.com      1
savanah.stracke@gmail.com     1
vivienne.fritsch@yahoo.com    1
abigayle.crooks@yahoo.com     1
Name: count, Length: 49833, dtype: Int64


In [18]:
#Lets check with one of the duplicated Emails
print(df[df['Email']=='marianne31@yahoo.com'])

      First Name Last Name                 Email Application Date    Country  \
3927     Arielle    Harris  marianne31@yahoo.com       2021-05-04    Morocco   
19463   Tristian   Quitzon  marianne31@yahoo.com       2021-04-13      Niger   
30935        Tod   Kuhlman  marianne31@yahoo.com       2019-11-17  Gibraltar   

       YOE  Seniority                         Technology  \
3927    29       Lead                Security Compliance   
19463   13       Lead                Security Compliance   
30935   11  Architect  Social Media Community Management   

       Code Challenge Score  Technical Interview Score  Hired  \
3927                      2                          1  False   
19463                     1                          7  False   
30935                     2                          2  False   

                     Technology Grouped  
3927   Systems & Network Administration  
19463  Systems & Network Administration  
30935                Marketing & Design  


The cleaning of Email is discarded, multiple candidates can use same Email to applicate

Technology Group

In [7]:
df["Technology"].unique()

array(['Data Engineer', 'Client Success', 'QA Manual',
       'Social Media Community Management', 'Adobe Experience Manager',
       'Sales', 'Mulesoft', 'DevOps', 'Development - CMS Backend',
       'Salesforce', 'System Administration', 'Security',
       'Game Development', 'Development - CMS Frontend',
       'Security Compliance', 'Development - Backend', 'Design',
       'Business Analytics / Project Management',
       'Development - Frontend', 'Development - FullStack',
       'Business Intelligence', 'Database Administration',
       'QA Automation', 'Technical Writing'], dtype=object)

The technologies can be grouped, looking for their similarities, this can help the visualization, mostly in the required pie chart

In [8]:
df["Technology Grouped"] = df["Technology"] #We dont replace directly in Technology column, that data can be usefull in a future

In [9]:
df['Technology Grouped'].replace({
    'Development - Backend': 'Development',
    'Development - Frontend': 'Development',
    'Development - FullStack': 'Development',
    'Development - CMS Backend': 'Development',
    'Development - CMS Frontend': 'Development',
    'Game Development': 'Development',
    'QA Manual': 'Quality Assurance',
    'QA Automation': 'Quality Assurance',
    'Data Engineer': 'Data Engineering',
    'Business Intelligence': 'Data Engineering',
    'Database Administration': 'Data Engineering',
    'System Administration': 'Systems & Network Administration',
    'Security': 'Systems & Network Administration',
    'Security Compliance': 'Systems & Network Administration',
    'Sales': 'Business & Sales',
    'Client Success': 'Business & Sales',
    'Business Analytics / Project Management': 'Business & Sales',
    'Social Media Community Management': 'Marketing & Design',
    'Design': 'Marketing & Design',
    'Adobe Experience Manager': 'Marketing & Design',
    'Mulesoft': 'Systems & Network Administration',
    'Salesforce': 'Business & Sales',
    'DevOps': 'Systems & Network Administration',
    'Technical Writing': 'Marketing & Design'
}, inplace=True)

print(df['Technology Grouped'].value_counts())

C:\Users\Usuario\AppData\Local\Temp\ipykernel_4672\2200183378.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Technology Grouped'].replace({


Technology Grouped
Development                         13447
Systems & Network Administration    11583
Marketing & Design                   7789
Business & Sales                     7569
Data Engineering                     5818
Quality Assurance                    3794
Name: count, dtype: int64


Object to string

Some columns are objects and can be changed to string to improve performance and reduce storage requirements

In [12]:
print(df.dtypes) #This code can be executed after and before the transformation to see the change

First Name                   string[python]
Last Name                    string[python]
Email                        string[python]
Application Date                     object
Country                      string[python]
YOE                                   int64
Seniority                    string[python]
Technology                   string[python]
Code Challenge Score                  int64
Technical Interview Score             int64
Hired                                  bool
Technology Grouped           string[python]
dtype: object


In [11]:
df['First Name'] = df['First Name'].astype('string')
df['Last Name'] = df['Last Name'].astype('string')
df['Email'] = df['Email'].astype('string')
df['Country'] = df['Country'].astype('string')
df['Seniority'] = df['Seniority'].astype('string')
df['Technology'] = df['Technology'].astype('string')
df['Technology Grouped'] = df['Technology Grouped'].astype('string')

Date format

Date is also an object, lets change it to datetime format

In [13]:
df['Application Date'] = pd.to_datetime(df['Application Date'], format='%Y-%m-%d')

Correlation

In [14]:
colmns_select = ["YOE","Code Challenge Score", "Technical Interview Score", "Hired"] #Numerical and Hired (Bool). The high ammount of categorical unique values would make the correlation really big
df_corr = df[colmns_select]


correlation = df_corr.corr()

correlation

,YOE,Code Challenge Score,Technical Interview Score,Hired
YOE,1.000000,0.005871,-0.005878,0.000192
Code Challenge Score,0.005871,1.000000,-0.002373,0.435115
Technical Interview Score,-0.005878,-0.002373,1.000000,0.431856
Hired,0.000192,0.435115,0.431856,1.000000


There is no strong correlation between the numerical columns. The two biggest value (0.43 approximately) are between Hired and Scores (Code Challenge and Tech Interview), this correlation isnt significant, and can it could be due to the fact that Hired is created based on the scores columns values.

Saving the cleaned dataframe

In [15]:
df.to_csv('data/candidates_clean.csv', index=False)

Migration to BD

it replaces the raw df to the cleaned version we get in this jupyternotebook in the BD

In [19]:
#This is very similar to migration code, just with a little changes
file_path = "data/candidates_clean.csv"  
candidates = pd.read_csv(file_path, delimiter=',') #In the clean csv the delimitier is , and not ;

engine = create_engine(f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}")

try:
    candidates.to_sql("candidates_clean", engine, if_exists="replace", index=False)
    print("Succesfull migration")
except Exception as e:
    print(f"Error in migration: {e}")

Succesfull migration
